In [ ]:
import quantumsim as qs

# This notebook demonstrates the use of the quantumsim.Controller

## Circuit definition

We start off be defining the circuits that will be used in our experiment. In this specific these corresponds to the implementation of a standard 3-qubit parity check experiment. We define the bell state preparation circuit and the ZZ and XX parity-check circuits. Importantly for these cirucits is the use of conditional rotation. For the bell-state preparation, a 90 degree rotation is applied to the second data qubit only in the ZZ-experiment, but not in the XX-experiment. Conditional feedback rotation are also used in the ZZ and XX parity check circuits to implement a conditional reset operation, based on the measurement outcome of the ancilla.

In [ ]:
bell_prep_circuit = (
    qs.gates.rotate_y('D0', angle=90) + 
    qs.gates.rotate_y('D1', angle='cond_rot')
).finalize()

zz_stabilizer_circuit = (
    qs.gates.rotate_y('Z', angle=90) + 
    qs.gates.cphase('D0', 'Z', angle=180) + 
    qs.gates.cphase('Z', 'D1', angle=180) +
    qs.gates.rotate_y('Z', angle=-90) +  
    qs.gates.measure('Z', result='meas_z') + 
    qs.gates.rotate_x('Z', angle='z_feedback_rot')
).finalize()

## Controller definition

The next step is to define the experiment controller. In specific we are interested in running two experiments - a ZZ-parity check experiment, where a pair of data-qubit is protected against bit-flip like errors and a interleved ZZ-XX type experiment, where the data qubits are protected against both bit and phase flip errors over a number of cycles.

The defintion of each experiment consists of defining a list of instructions to be executed by quantumsim and returned as a function. Each experiment must begin with a state preparation. We then apply the bell-state preparation circuit, followed by N rounds of repetaed parity-checks. The circuits are stored with a corresondping label, and this label is also used to apply the circuit. In our case, the three circuits we previously created will eventually be saved as "bell_prep", "zz_stabilizer" and "xx_stabilizer". 

The controller applied circuits to the stored state via controller.apply. When applying each circuit, the parameters of the cirucits can be directly given when initializing the experiment, done by the **params kwargs. Additional arguements can be provided to each of the apply circuits. In our case, passing the cycle number will automatically label the collected circuit outcomes by the cycle number.

The circuit outcome, returned from the apply function can be saved to a common dataset for the experiment. This is done by the to_dataset function. In this case, since the same circuit is repeatedly applied, we explcitly tell the controller to append these outcome to a common array, along the corresponding cycle number.

The current state stored by the controlled can also be saved. In our case, we are interested only in the diagonal elements of the density matrix, which we extract at the end of each cycle and add to the dataset.

In [ ]:
class ExampleController(qs.Controller):
    def zz_experiment(self, num_cycles=10):
        #the inner def can be skipped if you don't want to have a function which auomatically scans over num_cycles
        def experiment(**params):
            self.prepare_state()

            self.apply('bell_prep', **params)
            for cycle in range(num_cycles):
                self.to_dataset(self.apply('zz_stabilizer', cycle=cycle, **params), concat_dim='cycle')               
                self.state.renormalize()
                self.to_dataset(self.state.diagonal.assign_coords(cycle=cycle), concat_dim='cycle')
        return experiment

We finally initialize the controller, providing the circuits we created with their corresponding labels as well as the values for the conditional parameters of the circuits. Notice that we did not provide expression for the measurement outcomes, as a default born-projection rule is already implemented in the standard measurement operation in quantumsim. Each parameter can be defined by either an expression (a function) or a fixed value. These values can always be overwritten later on for convenience.

In [ ]:
controller = ExampleController(
    circuits={
        'bell_prep' : bell_prep_circuit,
        'zz_stabilizer' : zz_stabilizer_circuit
    },
    parameters={
        'z_feedback_rot': lambda outcome: 180 if outcome.sel(param='meas_z') == 1 else 0,
        'x_feedback_rot': lambda outcome: 180 if outcome.sel(param='meas_x') == 1 else 0,
        'cond_rot' : 90
    }
)

When defining the parameter definitio, there are a few special arguements supported by the controller. If a random number generator is required by the application, the one stored within the controller can be accessed by the rng arguement. Similarly, if the parameters depends on the current state stored within the decored, this can be acccessed via the state arguement. Since this is the full state of the system, the qubit indicies of the current operation are autmatically provided via the inds keyword. Finally, if the parameter depends on any of the previous parameters of the current circuit, these can be accessed by the outcome arguement. Note that these are stored as a xarry.DataArray.

## Running the experiment

Once we have defined everything, running an experiment is as simple as:

In [ ]:
outcome = controller.run(controller.zz_experiment(5), seed=range(5))

In [ ]:
outcome

Note in the function above we defined an experiment of 5 QEC cycles. We can run repetitions of a given circuits via the seed arguement. If any parameter of the cirucit requires a rng, this seed is used to initialize the random number generator stored within the controller, allowing the reproducability of simulations. On the other hand, if a random number generator is not required, one can still easily repet an experiment M times by providing range(M). Each run is labeled by the corresponding seed and individual experiment outcomes are concatenated along this dimension.

As previously mentioned we can easily overwrite some of the paramters definition give to the controller initiall. For example, we can quickly change the value of the conditonal rotation used in the bell preparation circuit

In [ ]:
another_outcome = controller.run(controller.zz_experiment(5), seed=range(5), cond_rot=0)

In [ ]:
another_outcome

Inspecting the dataset files, one can notice that the outcomes for the diagonals and each cirucit are saved as separate arrays within the dataset. The dimensions of these arrays correspond to the seed and cycle we previously defined. In the case of the circuits, each parameter is labeled as param. In the case that multiple parameterized cirucuits are ran, the param coordinate is shared among all circuits and will lead to some NaN values in the full arrays. Not sure if I should fix this by setting param to something like zz_stabilizer_param automatically.